In [17]:
import os
import sys
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

In [18]:
browser = Chrome()
delay = 3
base_url = 'https://cafe.naver.com/misinggo'

In [19]:
browser.get(base_url)
browser.implicitly_wait(delay)

In [20]:
browser.find_element_by_css_selector('span.gnb_txt').click()
browser.implicitly_wait(delay)

## 네이버 로그인 보안 강화로 코드로 접속 불가, 직접 쳐야한다.
#### browser.find_element_by_name('id').send_keys(naver_id)
#### browser.find_element_by_name('pw').send_keys(naver_pw)
#### browser.find_element_by_css_selector('input.btn_global').send_keys(Keys.ENTER)

In [21]:
cafe_menu = browser.find_element_by_css_selector('div#cafe-menu')
browser.implicitly_wait(delay)

In [22]:
menu_list = cafe_menu.find_elements_by_tag_name('a')

In [23]:
menu_source = pd.DataFrame({'name' : [], 'source' : []})
name = []
source = []

for menu in menu_list :
    menu_name = menu.text
    if '전체글' in menu_name : # 원하는 검색어를 집어넣어야 한다.
        insert_menu = pd.DataFrame({'name' : [menu_name], 'source' : [menu]})
        menu_source = menu_source.append(insert_menu)

In [24]:
menu_source.index = range(len(menu_source))

In [25]:
menu_source

,name,source
0,전체글보기,<selenium.webdriver.remote.webelement.WebEleme...


In [26]:
# 원하는 카테고리 클릭을 위해서 인덱싱을 진행하는데, 앞에부분만 바꾸면된다.
menu_source.iloc[0,1].send_keys(Keys.ENTER)
                                            
browser.implicitly_wait(delay)

In [27]:
i = 1
post_list = pd.DataFrame({'name' : [],
                         'user' : [],
                         'time' : [],
                         'link' : []})
while True :
    try :
        browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
        page = browser.find_element_by_css_selector('div.prev-next')
        page_num = page.find_elements_by_tag_name('a')
    except :
        page = browser.find_element_by_css_selector('div.prev-next')
        page_num = page.find_elements_by_tag_name('a')        
    
    for num in range(len(page_num)) :           
            try :  
                browser.switch_to_frame(browser.find_element_by_name('cafe_main'))
                page = browser.find_element_by_css_selector('div.prev-next')
                page_num = page.find_elements_by_tag_name('a')
                if '다음' in page_num[num].text :
                    page_num[num].send_keys(Keys.ENTER)
                    break
                elif '이전' in page_num[num].text :
                    pass
                else :
                    page_num[num].send_keys(Keys.ENTER)
            except :
                page = browser.find_element_by_css_selector('div.prev-next')
                page_num = page.find_elements_by_tag_name('a')
                if '다음' in page_num[num].text :
                    page_num[num].send_keys(Keys.ENTER)
                    break        
                elif '이전' in page_num[num].text :
                    pass
                else :
                    page_num[num].send_keys(Keys.ENTER)
                
            source = browser.page_source
            html = BeautifulSoup(source, 'html.parser')
            tbody = html.find_all('div', {'class' : 'article-board m-tcol-c'})
            post_name = tbody[1].find_all('a', {'class' :'article'})
            post_user = tbody[1].find_all('a', {'class' : 'm-tcol-c'})
            post_time = tbody[1].find_all('td', {'class' : 'td_date'})

            name = []
            user = []
            time = []
            link = []

            for n in post_name :
                n = n.text.replace('\n','')
                name.append(re.sub(r' {2,}' , ' ', n))
                
            for u in post_user :
                user.append(u.text)

            for t in post_time :
                time.append(re.sub('[^0-9\.\:]', '', t.text))
                
            for l in range(len(post_name)) :
                post_link = base_url + post_name[l]['href']
                link.append(post_link)

            posting = pd.DataFrame({'name' : name,
                                    'user' : user,
                                    'time' : time,
                                   'link' : link})

            post_list = post_list.append(posting)

    if len(page_num) < 11 :
        break
    
post_list = post_list.drop_duplicates(subset='name', keep='last')
post_list.index = range(len(post_list))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  


IndexError: list index out of range

In [28]:
post_list = post_list.drop_duplicates(subset='name', keep='last')
post_list.index = range(len(post_list))

post_list

,name,user,time,link
0,이곳은 큐브 입니다,클럽문의 01077994890,13:57,https://cafe.naver.com/misinggo/ArticleRead.nh...
1,이곳은 마운틴 입니다,클럽문의 01077994890,13:53,https://cafe.naver.com/misinggo/ArticleRead.nh...
2,이곳은 헤븐 입니다,클럽문의 01077994890,13:50,https://cafe.naver.com/misinggo/ArticleRead.nh...
3,이곳은 돔 입니다,클럽문의 01077994890,13:44,https://cafe.naver.com/misinggo/ArticleRead.nh...
4,장안동 다이마루공장 입니다,중고사서절약하자,10:34,https://cafe.naver.com/misinggo/ArticleRead.nh...
5,장안동 다이마루공장 거레처 구합니다,중고사서절약하자,09:48,https://cafe.naver.com/misinggo/ArticleRead.nh...
6,숙녀복 다이마루공장 입니다,중고사서절약하자,07:59,https://cafe.naver.com/misinggo/ArticleRead.nh...
7,악세사리 도매전문(귀걸이 도매),JM쥬얼리,2019.02.07.,https://cafe.naver.com/misinggo/ArticleRead.nh...
8,안녕하세요 콘솔지퍼 생산 공장 입니다.,JanganZipper,2019.02.07.,https://cafe.naver.com/misinggo/ArticleRead.nh...
9,미싱사(누비)구합니다 -주5일,낚시바람,2019.02.07.,https://cafe.naver.com/misinggo/ArticleRead.nh...


In [29]:
post_list.to_csv(os.getcwd() + "/" + "cafe_misinggo" + ".csv", mode = 'w', header = True, index = None, encoding = 'utf-8-sig')

In [30]:
post_list['user'].value_counts()

원도매 SY컴퍼니 대표        80
디브릿지01084770807     72
마유리                 61
리리링리리               58
인디언캠핑               49
도깨비이두표              37
더미더미                34
특수피혁 전문제조           31
흰소나무                28
쓰따로                 26
최선을다하자              26
모두스                 25
봉제사장                25
고집쎈                 25
작은별 12              25
kwan0906            24
BM의류                23
개장수네형               21
울프23                21
gsys1956            21
Skollbag            21
kaka0371            21
클럽문의 01077994890    21
4랑e아픈2u             20
iiiiiii             20
Anan Promotion      20
skzom               20
종아조아                19
쇼미더마니               19
광주해성                19
                    ..
오리엔탈시럽               1
와펜전문가                1
데나                   1
bdjkorea             1
도사                   1
마이니                  1
신성호                  1
ryry090              1
미싱의신                 1
rhckdtjr             1
날아라바둑이               1
파포파                  1
채화니        

In [31]:
post_list['user'].value_counts().to_csv(os.getcwd() + "/" + "cafe_misinggo_userlist" + ".csv", mode = 'w', header = True, encoding = 'utf-8-sig')